In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pybullet as p
import pybullet_data
import time
import math
import numpy as np
import json

from scripts.buffer import ReplayBuffer
from scripts.SAC import AgentSAC
from scripts.utils import diff_drive_control, get_state, compute_reward_done, get_spawn, get_goal, add_random_blocks

In [3]:
# DEFINE VARIABLES AND HYPERPARAMETERS
PI = math.pi
NUM_EPISODES = 2500
ACTION_DURATION = 0.5
STEP_ACTION_DUARTION = int(ACTION_DURATION * 240.0)
MAX_EPISODE_TIME = 20
MAX_STEPS_PER_EPISODE = int(MAX_EPISODE_TIME / ACTION_DURATION)
NEW_GOAL_EVERY = 2
SAVE_EVERY = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Load Environment
physicsClient = p.connect(p.GUI)
p.setGravity(0, 0, -9.8)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
room_size = 10
add_random_blocks(num_blocks=10, x_range=(-room_size, room_size), y_range=(-room_size, room_size))

p.loadURDF('plane.urdf')

init_pos = [0, 0, 0]
init_orientation = p.getQuaternionFromEuler([0, 0, 0])
robotId = p.loadURDF('diff_drive_robot/diff_drive_robot.urdf', basePosition=init_pos, baseOrientation=init_orientation)
p.changeDynamics(robotId, linkIndex=-1, lateralFriction=0.0, spinningFriction=0.0, rollingFriction=0.0)
p.changeDynamics(robotId, 0, lateralFriction=5.0)
p.changeDynamics(robotId, 1, lateralFriction=5.0)
num_joints = p.getNumJoints(robotId) - 1

In [5]:
batch_size = 128
agent = AgentSAC(state_dim=367, action_dim=2).to(device)
agent.load_state_dict(torch.load('saves/model_state_2499.pth'))
buffer = ReplayBuffer(capacity=10000, batch_size=batch_size)

reward_tracking = []
dist_goal_tracking = []
step_tracking = []

C:\Users\joaqu\AppData\Local\Temp\ipykernel_48020\3096984387.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent.load_state_dict(torch.load('saves/model_state_2499.pth

In [ ]:
agent.train()
for episode in range(NUM_EPISODES):
    
    # set spawn and goal
    if episode % NEW_GOAL_EVERY == 0:
        # set spawn
        spawn = get_spawn()
        new_position = [spawn[0], spawn[1], 0.05]
        new_orientation = p.getQuaternionFromEuler([0, 0, spawn[2]])
        # set goal
        goal = get_goal(new_position)

    # Reset position
    p.resetBasePositionAndOrientation(robotId, posObj=new_position, ornObj=new_orientation)
    # collect initial state
    state = get_state(robotId, goal)

    total_reward = 0
    done = False
    print(f'Started episode {episode}, spwan: [{spawn[0]},{spawn[1]}] goal {goal}')
    for step in range(MAX_STEPS_PER_EPISODE):

        # choose and perform action
        action, _ = agent.actor.sample(torch.tensor(state, dtype=torch.float32).to(device))
        action = action.detach().cpu().numpy()
        action_drive = diff_drive_control(linear_vel=action[0], angular_vel=action[1])
        p.setJointMotorControlArray(robotId, range(num_joints), controlMode=p.VELOCITY_CONTROL, targetVelocities=action_drive, forces=[100, 100])
        start_time = time.time()
        for _ in range(STEP_ACTION_DUARTION):
            p.stepSimulation()
            time.sleep(1.0 / 240.0)
        # p.setJointMotorControlArray(robotId, range(num_joints), controlMode=p.VELOCITY_CONTROL, targetVelocities=[0, 0], forces=[100, 100])
        # for _ in range(24):
        #     p.stepSimulation()
        #     #time.sleep(1.0 / 240.0)

        # collect new state
        next_state = get_state(robotId, goal)

        # get reward and done
        reward, done = compute_reward_done(state, action, next_state)
        total_reward += reward

        # store transition in buffer
        buffer.push(state, action, next_state, reward, done)

        # update model
        if len(buffer) >= batch_size:
            # sample from batch
            states, actions, next_states, rewards, dones = buffer.sample()
            # update critic 
            agent.update_critic(states, actions, next_states, rewards, dones)
            # update actor
            agent.update_actor(states)
            
        # break if done
        if done:
            break

        # update state
        state = next_state
    
    robot_pos, _ = p.getBasePositionAndOrientation(robotId)
    dist_to_goal = np.sqrt((robot_pos[0] - goal[0])**2  + (robot_pos[1] - goal[1])**2)

    # Results log
    print(f'Finished episode {episode} with total reward {total_reward}, average reward {total_reward / (step+1)}, distance to goal: {dist_to_goal}, in {step+1} steps')
    reward_tracking.append(total_reward)
    dist_goal_tracking.append(dist_to_goal)
    step_tracking.append(step)

    # Save model every few steps
    if episode % SAVE_EVERY == 0 or episode == NUM_EPISODES-1:
        torch.save(agent.state_dict(), f'saves/model_state_{episode}.pth')

# Save training data
with open('training_log/training_log_lidar.json', 'w') as file:
    json.dump({'Reward': reward_tracking, 'Distance': dist_goal_tracking, 'Steps': step_tracking}, file)

Started episode 0, spwan: [0,0] goal [0.807541416094618, 0.5322430047179205]
Finished episode 0 with total reward 1230.987681915679, average reward 205.16461365261318, distance to goal: 0.0357600712945101, in 6 steps
Started episode 1, spwan: [0,0] goal [0.807541416094618, 0.5322430047179205]
